<a href="https://www.kaggle.com/code/sohanamitarathod/airline-passengers-prediction-using-gru-rnn?scriptVersionId=141461049" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

The dataset provides a record of the number of people travelling in US airlines in a particular month. It has a record of 142 months. It has 2 columns “Month” and “No. of Passengers”. But in this case, I want to use univariate dataset. Only “No. of Passengers” is used.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM,GRU,SimpleRNN
from sklearn.preprocessing import MinMaxScaler

In [ ]:

df=pd.read_csv("/kaggle/input/passenger-prediction-using-gru-rnn/AirPassengers.csv")
df.head()


In [ ]:
df.drop(['Month'],axis=1,inplace=True)
dataset = np.array(df)
dataset.reshape(-1,1)

It shows that the number of passengers is linearly increasing over the months.

Machine learning model/ Neural network works better if all the data is scaled.



In [ ]:
scaler = MinMaxScaler()
dataset = scaler.fit_transform(dataset)


Divide the data for training and testing. split the dataset into (75% training and 25% testing). In the dataset, we can estimate the ‘i’th value based on the ‘i-1’th value. You can also increase the length of the input sequence by taking i-1,i-2,i-3… to predict ‘i’th value

In [ ]:
train_size = int(len(dataset) * 0.75)
test_size = len(dataset) - train_size
train=dataset[:train_size,:]
test=dataset[train_size:142,:]
def getdata(data,lookback):
    X,Y=[],[]
    for i in range(len(data)-lookback-1):
        X.append(data[i:i+lookback,0])
        Y.append(data[i+lookback,0])
    return np.array(X),np.array(Y).reshape(-1,1)
lookback=1
X_train,y_train=getdata(train,lookback)
X_test,y_test=getdata(test,lookback)
X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1],1)

In [ ]:
model=Sequential()
model.add(GRU(5,input_shape=(1,lookback)))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [ ]:
model.summary()

Now, the model is ready. So start training the model.

In [ ]:
model.fit(X_train, y_train, epochs=50, batch_size=1)
y_pred=model.predict(X_test)
y_test=scaler.inverse_transform(y_test)
y_pred=scaler.inverse_transform(y_pred)

In [ ]:
plt.figure(figsize=(14,5))
plt.plot(y_test, label = 'Real number of passengers')
plt.plot(y_pred, label = 'Predicted number of passengers')
plt.ylabel('# passengers')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test,y_pred)